# Explore multiple datasets

In this notebook, we are going to experiment with characterising the three datasets that we have in terms of data quality and demographic characteristics.

This notebook is intended to be run on the exported, federated csv file. The file should be exported using `Federating and saving multiple datasets.ipynb`

### First, we read the data and extract the most common purpose labels

In [ ]:
import pandas as pd
import numpy as np
import geojson as gj
import sklearn.cluster as sc
import sklearn.metrics.pairwise as smp
import sklearn.metrics as sm

In [ ]:
import json
import copy

In [ ]:
import folium
import branca.element as bre

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import seaborn as sns

In [ ]:
from IPython import display
from uuid import UUID

import bson.json_util as bju
import bson.objectid as boi

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import emission.analysis.modelling.tour_model.similarity as eamts

In [ ]:
import emission.core.wrapper.entry as ecwe
import emission.core.wrapper.confirmedtrip as ecwct

### Read data and setup variables

In [ ]:
all_expanded_df = pd.read_json(open("/tmp/federated_trip_only_dataset.json"), orient="records", typ="frame")
for id_col in ["_id", "raw_trip", "start_place", "end_place", "cleaned_trip"]:
    all_expanded_df[id_col] = all_expanded_df[id_col].apply(lambda i: boi.ObjectId(i["$oid"]))
    
all_expanded_df["user_id"] = all_expanded_df["user_id"].apply(lambda u: UUID(u["$uuid"]))
all_expanded_df.tail()

In [ ]:
all_expanded_df.columns

In [ ]:
def get_unique_program(user_id):
    all_programs = all_expanded_df[all_expanded_df.user_id == user_id]["program"].unique()
    assert len(all_programs) == 1, f"all_programs = {all_programs}"
    return all_programs[0]

participant_df = pd.DataFrame(all_expanded_df.user_id.unique(), columns=["user_id"])
participant_df = participant_df[participant_df.user_id != 0]
participant_df.set_index("user_id", inplace=True, drop=True)
participant_df["program"] = [get_unique_program(u) for u in participant_df.index]
participant_df

In [ ]:
modeling_support_objects = {}

In [ ]:
FINAL_RADIUS = 500
FINAL_FILTER_OURSIM = False
FINAL_CUTOFF_OURSIM = False
FINAL_POINT_DBSCAN = sc.DBSCAN(FINAL_RADIUS, min_samples=2, metric="precomputed")
FINAL_TRIP_DBSCAN = sc.DBSCAN(FINAL_RADIUS * 2, min_samples=2, metric="precomputed")

### Standard functions (currently copied over from other notebooks; should be refactored into a python file)

In [ ]:
def get_loc_df(loc_series):
    loc_df = pd.DataFrame(loc_series.apply(lambda p: p["coordinates"]).to_list(), columns=["longitude", "latitude"])
    # display.display(end_loc_df.head())
    return loc_df

In [ ]:
def get_distance_matrix(loc_df):
    EARTH_RADIUS = 6371000
    radians_lat_lon = np.radians(loc_df[["latitude", "longitude"]])
    dist_matrix_meters = pd.DataFrame(smp.haversine_distances(radians_lat_lon, radians_lat_lon) * 6371000)
    return dist_matrix_meters

In [ ]:
def add_loc_clusters(user_id, modeling_support_objects, trip_df):
    user_trip_df = trip_df[trip_df.user_id == user_id]
    start_distance_matrix = get_distance_matrix(get_loc_df(user_trip_df.start_loc))
    end_distance_matrix = get_distance_matrix(get_loc_df(user_trip_df.end_loc))
    start_loc_model = copy.copy(FINAL_POINT_DBSCAN).fit(start_distance_matrix)
    end_loc_model = copy.copy(FINAL_POINT_DBSCAN).fit(end_distance_matrix)
    trip_df.loc[user_trip_df.index, "start_loc_cluster"] = start_loc_model.labels_
    trip_df.loc[user_trip_df.index, "end_loc_cluster"] = end_loc_model.labels_

    curr_model_support = modeling_support_objects.get(user_id)
    if curr_model_support is None:
        modeling_support_objects[user_id] = {}
        curr_model_support = modeling_support_objects[user_id]
    curr_model_support["start_distance_matrix"] = start_distance_matrix
    curr_model_support["end_distance_matrix"] = end_distance_matrix   
    curr_model_support["start_loc_model"] = start_loc_model
    curr_model_support["end_loc_model"] = end_loc_model

    return trip_df

In [ ]:
def add_trip_clusters_dbscan(user_id, trip_df):
    user_trip_df = trip_df[trip_df.user_id == user_id]
    all_combos = user_trip_df.groupby(["start_loc_cluster", "end_loc_cluster"])
    valid_combos = [p for p in all_combos.groups if p[0] != -1 and p[1] != -1]
    print(f"After validating, all_combos {len(all_combos.groups)} -> {len(valid_combos)}")
    all_combos_dict = dict(all_combos.groups)
    valid_combos_series = pd.Series(valid_combos)
    for g, idxlist in all_combos_dict.items():
        print(g, idxlist)
        match = valid_combos_series[valid_combos_series == g]
        if len(match) == 0:
            print(f"invalid combo {g} found for entries {idxlist}, trip is not in a cluster")
            trip_df.loc[idxlist, "trip_cluster_dbscan"] = -1
        else:
            print(f"valid combo {g} found for entries {idxlist}, setting trip cluster to {match.index[0]}")
            trip_df.loc[idxlist, "trip_cluster_dbscan"] = int(match.index[0])
    return trip_df

In [ ]:
def add_ground_truth(trip_df, columns, gt_label):
    unique_tuples = dict(trip_df.groupby(by=columns).groups)
    for i, idxlist in enumerate(unique_tuples.values()):
    # print(i, idxlist)
        trip_df.loc[idxlist, gt_label] = i

In [ ]:
def add_trip_clusters_oursim(user_id, modeling_support_objects, trip_df):
    user_trip_df = trip_df[trip_df.user_id == user_id]
    user_trip_list = [ecwe.Entry({"data": ecwct.Confirmedtrip(tr), "_id": tr["_id"], "metadata": {"key": "analysis/confirmed_trip"}}) for tr in user_trip_df.to_dict("records")]
    curr_sim = eamts.similarity(user_trip_list, FINAL_RADIUS, shouldFilter=FINAL_FILTER_OURSIM, cutoff=FINAL_CUTOFF_OURSIM)
    curr_sim.fit()
    trip_df.loc[user_trip_df.index, "trip_cluster_oursim"] = curr_sim.labels_.to_list()
    modeling_support_objects[user_id]["similarity_model"] = curr_sim
    return trip_df

In [ ]:
def h_score_no_na(labels_true, labels_pred):
    na_index = labels_true[pd.isna(labels_true)].index
    # Before we set the index to nan; we don't want to have a side effect here!
    new_labels_pred = labels_pred.copy()
    new_labels_pred.loc[na_index] = np.nan
    if (len(na_index) > 0):
        print(f"Dropping nan indices {na_index} before calculating score")
        # print(f"{labels_true.dropna()}, {new_labels_pred.dropna()}")
    return sm.homogeneity_score(labels_true.dropna(), new_labels_pred.dropna())

In [ ]:
def request_count(labels_pred):
    # once per real cluster
    # once per noisy point (since it is not in a cluster)
    # once per filtered trip (not really necessary for our current regime, but good to be prepared)
    return np.count_nonzero(labels_pred.unique() >= 0) \
                    + np.count_nonzero(labels_pred == -1) \
                    + np.count_nonzero(labels_pred == -2)

In [ ]:
def update_basic_stats(user_id, participant_df, trip_df):
    user_trip_df = trip_df[trip_df.user_id == user_id]
    basic_stats = {}
    basic_stats["n_labeled_trips"] = len(user_trip_df)
    basic_stats["unique_label_combos"] = list(user_trip_df.groupby(["mode_confirm", "purpose_confirm", "replaced_mode"]).groups)
    basic_stats["start_loc_in_cluster"] = np.count_nonzero(user_trip_df.start_loc_cluster != -1)
    basic_stats["end_loc_in_cluster"] = np.count_nonzero(user_trip_df.end_loc_cluster != 1)
    for algo in ["dbscan", "oursim"]:
        print(f"Generating basic stats for {user_id} {len(user_trip_df)}, algo {algo} ")
        basic_stats[f"n_trip_in_cluster_{algo}"] = np.count_nonzero(user_trip_df[f"trip_cluster_{algo}"] != -1)
        # need to add one to the max, since the cluster labels start at 0
        basic_stats[f"n_clusters_{algo}"] = user_trip_df[f"trip_cluster_{algo}"].max() + 1
        basic_stats[f"cluster_trip_ratio_{algo}"] = basic_stats[f"n_clusters_{algo}"] / basic_stats[f"n_trip_in_cluster_{algo}"]
        basic_stats[f"homogeneity_score_{algo}"] = h_score_no_na(user_trip_df.ground_truth_by_tuple, user_trip_df[f"trip_cluster_{algo}"])
        basic_stats[f"request_count_{algo}"] = request_count(user_trip_df[f"trip_cluster_{algo}"])
        basic_stats[f"request_pct_{algo}"] = basic_stats[f"request_count_{algo}"] / basic_stats["n_labeled_trips"]

    # print(f"Adding cols {basic_stats.keys()} with vals {basic_stats.values()}")
    participant_df.loc[user_id, basic_stats.keys()] = basic_stats.values()
    return participant_df

Target exploratory analysis:

- number of users
- number of trips
- labeled trip/user distribution
- number of unique combinations of labels
- distribution of unique combination of labels (overall)
- distribution of unique combination of labels (per-user)
- number of trips whose end point is in a cluster
- number of trips whose start point is in a cluster
- number of trips where trip is in a cluster
- number of clusters

In [ ]:
for u in participant_df.index:
    all_expanded_df = add_trip_clusters_dbscan(u, add_loc_clusters(u, modeling_support_objects,all_expanded_df))
    add_trip_clusters_oursim(u, modeling_support_objects, all_expanded_df)
    add_ground_truth(all_expanded_df, ["mode_confirm", "purpose_confirm", "replaced_mode"], "ground_truth_by_tuple")

In [ ]:
for u in participant_df.index:
    update_basic_stats(u, participant_df, all_expanded_df)

### Again, let's focus on one dataset before generalizing to other datasets

In [ ]:
minipilot_df = participant_df[participant_df.program == "minipilot"]
minipilot_df.head(n=2)

In [ ]:
minipilot_df[["n_labeled_trips", "start_loc_in_cluster", "end_loc_in_cluster", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim"]].plot(kind="bar", figsize=(20,5))

# Final results, generalized to the entire dataset



### First, let's just display everything, without grouping by program

In [ ]:
participant_df.columns

In [ ]:
ax = participant_df[["n_labeled_trips", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim", "request_count_dbscan", "request_count_oursim"]].plot(kind="bar", use_index=False, figsize=(30,10))

### Next, let's group by dataframe to see if there are consistent program level differences

In [ ]:
participant_df[participant_df.program == "minipilot"][["n_labeled_trips", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim", "request_count_dbscan", "request_count_oursim"]].plot(kind="bar", figsize=(20,5), use_index=False)

In [ ]:
participant_df[participant_df.program == "nrel_lh"][["n_labeled_trips", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim", "request_count_dbscan", "request_count_oursim"]].plot(kind="bar", figsize=(20,5), use_index=False)

In [ ]:
participant_df[participant_df.program == "stage"][["n_labeled_trips", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim", "request_count_dbscan", "request_count_oursim"]].plot(kind="bar", figsize=(20,5), use_index=False)

### Assessing clustering effectiveness

We use our standard metrics to assess the tradeoff between request pct and homogeneity score.

In [ ]:
fig = plt.Figure(figsize=(20,5))
axarr = fig.subplots(1,2)
colors = plt.get_cmap("Accent", 4).colors

for i, algo in enumerate(["dbscan", "oursim"]):
    ax = axarr[i]
    # ax = result_df.plot.scatter(x=f"no_filter_no_cutoff_{r}_homogeneity_score_tuple", y=f"no_filter_no_cutoff_{r}_request_pct", label=f"no_filter_no_cutoff_{r}")
    for j, p in enumerate(participant_df.program.unique()):
        curr_p_df = participant_df[participant_df.program==p]
        curr_p_df.plot.scatter(x=f"homogeneity_score_{algo}", y=f"request_pct_{algo}", color=colors[j], label=f"{p}", ax=ax)
    ax.set_xlabel("homogeneity score")
    ax.set_ylabel("request pct")
fig

The request percents on for both the DBSCAN (left) and oursim (right) are similar, but the homogeneity for oursim is clearly better. In fact, we can see a clear linear trend in which a higher request pct leads to better scores. Again, this is an indication that single trip clusters are dominating our results. This is also likely to involve some outliers in which we have very little data to work with.

Let's re-plot with the number of trips to get a sense of that correlation

In [ ]:
fig = plt.Figure(figsize=(20,5))
axarr = fig.subplots(1,2)
colors = plt.get_cmap("Accent", 4).colors

for i, algo in enumerate(["dbscan", "oursim"]):
    ax = axarr[i]
    participant_df.plot.scatter(x=f"homogeneity_score_{algo}", y=f"request_pct_{algo}", c="n_labeled_trips", cmap="viridis", ax=ax)
    ax.set_xlabel("homogeneity score")
    ax.set_ylabel("request pct")
fig

Since we appear to have a linear relationship between the h-score and the request pct, let's use one of the axes for the number of trips, to see if the visualization becomes more clear.

In [ ]:
fig = plt.Figure(figsize=(20,5))
axarr = fig.subplots(1,2)
colors = plt.get_cmap("Accent", 4).colors

for i, algo in enumerate(["dbscan", "oursim"]):
    ax = axarr[i]
    for j, p in enumerate(participant_df.program.unique()):
        curr_p_df = participant_df[participant_df.program==p]
        curr_p_df.plot.scatter(x=f"n_labeled_trips", y=f"request_pct_{algo}", ax=ax, color=colors[j], label=p)
    ax.set_xlabel("number of labeled trips")
    ax.set_ylabel("request pct")
fig

The linear scale is resulting in a bunch of points clustered around the edge of the X-axis, with little differentiation between them. Let's try a log scale instead to see if it makes more sense.

In [ ]:
fig = plt.Figure(figsize=(20,5))
axarr = fig.subplots(1,2)
colors = plt.get_cmap("Accent", 4).colors

for i, algo in enumerate(["dbscan", "oursim"]):
    ax = axarr[i]
    for j, p in enumerate(participant_df.program.unique()):
        curr_p_df = participant_df[participant_df.program==p]
        curr_p_df.plot.scatter(x=f"n_labeled_trips", y=f"request_pct_{algo}", ax=ax, color=colors[j], label=p, logx=True)
    ax.set_xlabel("number of trips (log scale)")
    ax.set_ylabel("request pct")
fig

This does appear to show a reasonably clear trend. If we have less than 10 trips, we cannot see any patterns; request_pct = 1. Between 10 and 100 trips, there is a weak trend towards more data providing better results, but it appears to be linear on the log scale, so there is likely some diminishing returns.

Speaking of trends, let us look at more statistical visualizations of these results.

In [ ]:
fig = plt.Figure(figsize=(10,10))
axarr = fig.subplots(2,2,sharex=True, sharey=True)
participant_df.boxplot("request_pct_dbscan", by="program", ax=axarr[0][0])
participant_df.boxplot("request_pct_oursim", by="program", ax=axarr[0][1])
participant_df.boxplot("homogeneity_score_dbscan", by="program", ax=axarr[1][0])
participant_df.boxplot("homogeneity_score_oursim", by="program", ax=axarr[1][1])
fig

Again, the request percentages for the programs are similar, but the homogeneity score is significantly better for oursim. Based on the results above, let's plot separately for nTrips > 10 and nTrips > 100, for oursim.

In [ ]:
fig = plt.Figure(figsize=(10,10))
axarr = fig.subplots(2,2,sharex=True, sharey=True)
participant_df[participant_df.n_labeled_trips > 10].boxplot("request_pct_oursim", by="program", ax=axarr[0][0])
axarr[0][0].set_title("num labeled trip > 10")
participant_df[participant_df.n_labeled_trips > 100].boxplot("request_pct_oursim", by="program", ax=axarr[0][1])
axarr[0][1].set_title("num labeled trip > 100")
participant_df[participant_df.n_labeled_trips > 10].boxplot("homogeneity_score_oursim", by="program", ax=axarr[1][0])
axarr[1][0].set_title("num labeled trip > 10")
participant_df[participant_df.n_labeled_trips > 100].boxplot("homogeneity_score_oursim", by="program", ax=axarr[1][1])
axarr[1][1].set_title("num labeled trip > 100")
fig

As we can see, that tightens up the results considerably. There are now no outliers, and the IQR is also smaller. It looks like we end up with a mean request % between 0.3 and 0.5 and a mean h-score between 0.5 and 0.7, which seems pretty decent.

### Old estimates using cluster count

Finally, I want to spend a little bit of time focusing on the difference between the request % and the cluster trip ratio to fully understand what is going on. 

I originally came up with the cluster_trip ratio as an evaluation of the "tightness" of the cluster. Trips with a DBSCAN clustering are basically either in a cluster or noise. I found the ratio of the number of clusters to the number of trips in a cluster. If this was low, we would expect few requests, since most of the trips would be squished into a few clusters.

However, the cluster trip ratio boxplot for DBSCAN looks a lot better than the request %. I originally thought that this may be because there might be a lot of noisy trips; note that the cluster trip ratio does not include trips that are *not* in clusters, while the request percentage does.

However, if there are a lot of trips that are not in clusters, then the number of trips *in* the cluster should go down, so the denominator of the cluster_trip_ratio goes down, which increases the ratio. So why isn't that happening?

In [ ]:
# participant_df["cluster_trip_ratio"] = participant_df["n_clusters_dbscan"] / participant_df["trip_in_cluster_dbscan"]

In [ ]:
fig = plt.Figure(figsize=(10,3))
axarr = fig.subplots(1,2,sharey=True)
participant_df.boxplot("cluster_trip_ratio_dbscan", by="program", ax=axarr[0])
participant_df.boxplot("cluster_trip_ratio_oursim", by="program", ax=axarr[1])
fig

The NREL LH program does in fact have a better cluster ratio overall than the other two programs. But even in the other two programs, most of the ratios are pretty low. Still, we can't help everybody, and there are going to be a small number of people who are going to have to label more than half their trips. Still, it is gratifying to see that the max overall is just a bit higher than 0.7.

The same data with a slightly different visualization.

In [ ]:
# using plt.scatter here instead of pandas.plot since it is non-trivial to use the index as the x axis
# https://stackoverflow.com/questions/49834883/scatter-plot-form-dataframe-with-index-on-x-axis
# x=df.index does not work for me, may be due to an older version of pandas
color_list = plt.get_cmap("Accent", 3).colors
fig = plt.Figure(figsize=(10,5))
ax = fig.subplots(1,1)
for i, p in enumerate(participant_df.program.unique()):
    curr_p_df = participant_df[participant_df.program==p]
    ax.scatter([str(u) for u in curr_p_df.index], curr_p_df["cluster_trip_ratio_dbscan"], color=color_list[i], label=p)
ax.set_xticklabels(range(0,len(participant_df)))
ax.legend()
fig

### Entries with low cluster trip ratio and high request pct

In [ ]:
fig = plt.Figure(figsize=(20,5))
axarr = fig.subplots(1,2)
colors = plt.get_cmap("Accent", 4).colors

for i, algo in enumerate(["dbscan", "oursim"]):
    ax = axarr[i]
    for j, p in enumerate(participant_df.program.unique()):
        curr_p_df = participant_df[participant_df.program==p]
        # Focus on trips with > 10 entries for meaningful results
        curr_p_df[curr_p_df.n_labeled_trips > 10].plot.scatter(x=f"request_pct_{algo}", y=f"cluster_trip_ratio_{algo}", ax=ax, color=colors[j], label=p)
fig

While the oursim is essentially a straight line, there are more outliers for DBSCAN.
let's look at some of the outliers for DBSCAN.

In [ ]:
participant_df.query("request_pct_dbscan > 0.5 and cluster_trip_ratio_dbscan < 0.3 and n_labeled_trips > 10")[["program", "n_labeled_trips", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "request_pct_dbscan", "cluster_trip_ratio_dbscan"]]

Looking at the entries with the biggest differences:
- **71f21d53**:
  - clusters (2) + trips outside cluster (34) = 36. So request pct is 36/48 = 0.75
  - clusters (2) / trips in cluster (14) = 0.14
  
So basically, the original assumption was correct; it is the noisy points that make a difference. Basically, this happens in the case where there are many noisy clusters, but the points that are within the clusters, compress down well. In that case, the noisy trips carry a lot more weight in the request pct, and excluding them has a much higher impact.

- **a3587c47**:
  - clusters (2) + trips outside cluster (5) = 7. So request pct should be 7/12 = 0.58. **Why is this 0.66 instead?!**
  - clusters (2) / trips in cluster (7) = 0.28
  
There was a mistake in the n_clusters calculation. I was using the max of the column, but the labels start from zero, so the number of clusters is actually the max + 1.

Fixed in notebook. New calculation is:
- **a3587c47**:
  - clusters (3) + trips outside cluster (5) = 8. So request pct should be 8/12 = 0.66.
  - clusters (3) / trips in cluster (7) = 0.42
  
- **71f21d53**:
  - clusters (3) + trips outside cluster (34) = 37. So request pct is 37/48 = 0.77
  - clusters (3) / trips in cluster (14) = 0.21
  
Let's verify this by plotting with the difference in trips as the hue

In [ ]:
fig = plt.Figure(figsize=(5,5))
ax = fig.subplots(1,1)
colors = plt.get_cmap("Accent", 4).colors

temp_p_df = participant_df.copy()
temp_p_df["noisy_trip_pct"] = temp_p_df.apply(lambda r: r["n_labeled_trips"] - r["n_trip_in_cluster_dbscan"], axis=1) / temp_p_df["n_labeled_trips"]
temp_p_df.plot.scatter(x=f"request_pct_dbscan", y=f"cluster_trip_ratio_dbscan", c="noisy_trip_pct", cmap="viridis", ax=ax)
ax.set_xlabel("homogeneity score")
ax.set_ylabel("request pct")
fig

Alas, not sure that visualization is very useful :(

## DBSCAN-only plots, do not use

These are plots that were primarily created when we were looking at the DBSCAN clustering; they can still be used, but  some of the columns may not make as much sense any more.

In [ ]:
participant_df[participant_df.program == "minipilot"][["n_labeled_trips", "start_loc_in_cluster", "end_loc_in_cluster", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim"]].plot(kind="bar", figsize=(20,5), use_index=False)

In [ ]:
participant_df[participant_df.program == "nrel_lh"][["n_labeled_trips", "start_loc_in_cluster", "end_loc_in_cluster", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim"]].plot(kind="bar", figsize=(20,5), use_index=False)

In [ ]:
participant_df[participant_df.program == "stage"][["n_labeled_trips", "start_loc_in_cluster", "end_loc_in_cluster", "n_trip_in_cluster_dbscan", "n_clusters_dbscan", "n_clusters_oursim"]].plot(kind="bar", figsize=(20,5), use_index=False)